<a href="https://colab.research.google.com/github/Sweetmoon11/Machine-learning-for-spinal-diagnostic/blob/main/Exploracion_de_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

IMPORTACION DE DATOS

In [ ]:
import warnings, requests, zipfile, io
warnings.simplefilter('ignore')
import pandas as pd
from scipy.io import arff

In [ ]:
f_zip = 'http://archive.ics.uci.edu/ml/machine-learning-databases/00212/vertebral_column_data.zip'
r = requests.get(f_zip, stream=True)
Vertebral_zip = zipfile.ZipFile(io.BytesIO(r.content))
Vertebral_zip.extractall()

In [ ]:
data = arff.loadarff('column_2C_weka.arff')
df = pd.DataFrame(data[0])

Revision de datos importados

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df.dtypes

Aqui se va a revisar la estadistica de la primera columna

In [ ]:
df['pelvic_incidence'].describe()

Se revisa tambien la columna llamada lumbar_lordosis_angle

In [ ]:
df['lumbar_lordosis_angle'].describe()

Correspondiente a esto podriamos verificar varias cosas miremos una matriz de correlacion

In [ ]:
df[['pelvic_incidence', 'lumbar_lordosis_angle']].corr()

esto puede ser util para lo siguiente para reducir dimensionalidad


igualmente vamos a verlo de manera grafica en otros ambitos

 representa una variable numérica del DataFrame


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
df.plot()

Podemos tener en cuenta lo siguiente
- Las curvas de pelvic_incidence, lumbar_lordosis_angle y sacral_slope parecen moverse juntas en varios tramos.
- Esto refuerza la correlación positiva que ya se habia indicado (0.717 entre las dos primeras).


lo que vemos aca es la parte grafica referente a la relacion que hablabamos anteriormente


In [ ]:
import seaborn as sns
sns.scatterplot(x=df['pelvic_incidence'], y=df['lumbar_lordosis_angle'])
plt.title('Relación entre Pelvic Incidence y Lumbar Lordosis Angle')
plt.show()


podriamos tener en cuenta lo siguiente

- Los puntos están bastante agrupados en una diagonal ascendente, lo que sugiere que la relación es consistente.
- No parece haber muchos valores extremos o dispersos, aunque podrías verificarlo con un análisis de outliers.


In [ ]:
df.plot(kind='density',subplots=True,layout=(4,2),figsize=(12,12),sharex=False)
plt.show()

Aqui se puede ver unos graficos de densidad se pueden entender varias cosas

- Todas las variables tienen una sola “montaña” o pico, lo que indica que los datos están concentrados en torno a un valor central.
- Esto es bueno para modelos estadísticos, ya que evita complicaciones por distribuciones multimodales.


In [ ]:
df['degree_spondylolisthesis'].plot.density()

con lo siguiente podriamos indicar lo siguiente
- La forma de la curva sugiere una asimetría positiva (sesgo a la derecha).
- Esto significa que hay pocos pacientes con grados muy altos de espondilolistesis, mientras que la mayoría tiene valores bajos o normales.


In [ ]:
df['degree_spondylolisthesis'].plot.hist()

Aquí, la asimetría positiva (sesgo a la derecha) y la baja frecuencia de valores altos sugieren que esos puntos extremos no siguen el patrón general.


verificamos la distribucion

In [ ]:
df['class'].value_counts()

Aqui para que intentar obtener mas daros se va a convertir la columna en un valor numerico

In [ ]:
class_mapper = {b'Abnormal':1,b'Normal':0}
df['class']=df['class'].replace(class_mapper)

In [ ]:
df.plot.scatter(y='degree_spondylolisthesis',x='class')

Ahora vamos a tratar con el uso de las celdas anterores determinar como se corresponden los valores de otras caracteristicas con el objetivo

In [ ]:
features = ['pelvic_incidence', 'pelvic_tilt', 'lumbar_lordosis_angle',
            'sacral_slope', 'pelvic_radius', 'degree_spondylolisthesis']

for feature in features:
    plt.figure(figsize=(6, 4))
    sns.boxplot(x='class', y=feature, data=df)
    plt.title(f'{feature} vs class')
    plt.show()

Estas variables podrían ser predictoras fuertes en un modelo de clasificación.

In [ ]:
df.groupby('class')[features].mean().T

tienen diferencias marcadas entre clases. Esto tambien puede ser predictoras para un modelo de clasificacion


Calcula la matriz de correlación entre todas las variables numéricas y ordena las correlaciones respecto a la clase.

- Sirve para identificar qué variables están más relacionadas con el objetivo.
- Las correlaciones cercanas a ±1 indican relaciones fuertes.
- Puedes usar esto para seleccionar variables relevantes o detectar redundancia.


In [ ]:
df.groupby('class').boxplot(fontsize=20,rot=90,figsize=(20,10),patch_artist=True)

In [ ]:
corr_matrix = df.corr()
corr_matrix["class"].sort_values(ascending=False)

In [ ]:
pd.plotting.scatter_matrix(df,figsize=(12,12))
plt.show()

In [ ]:
# Plot figsize
fig, ax = plt.subplots(figsize=(10, 10))
# Generate Color Map
# colormap = sns.diverging_palette(220, 10, as_cmap=True)
colormap = sns.color_palette("BrBG", 10)
# Generate Heat Map, allow annotations and place floats in map
sns.heatmap(corr_matrix, cmap=colormap, annot=True, fmt=".2f")
#ax.set_yticklabels(column_names);
plt.show()